# Propeller model WPCC

In [ ]:
# %load imports.py
%load_ext autoreload
%autoreload 2
%reload_kedro
%config Completer.use_jedi = False  ## (To fix autocomplete)
%matplotlib inline

import warnings
warnings.filterwarnings('ignore')

import pandas as pd
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
from src.models.vmm import ModelSimulator
import matplotlib.pyplot as plt
from src.visualization.plot import track_plots, plot, captive_plot
import kedro
import numpy as np
import os.path
import anyconfig

import matplotlib
matplotlib.rcParams["figure.figsize"] = (10,10)
from src.symbols import *

# Read configs:
conf_path = os.path.join("../conf/base/")
runs_globals_path = os.path.join(
    conf_path,
    "runs_globals.yml",
)

runs_globals = anyconfig.load(runs_globals_path)
model_test_ids = runs_globals["model_test_ids"]

join_globals_path = os.path.join(
    conf_path,
    "join_globals.yml",
)

joins = runs_globals["joins"]
join_runs_dict = anyconfig.load(join_globals_path)

globals_path = os.path.join(
    conf_path,
    "globals.yml",
)
global_variables = anyconfig.load(globals_path)



vmm_names = global_variables["vmms"]

from wPCC_pipeline.pipelines.motion_regression.nodes import predict_force, fit_motions, create_model_from_motion_regression, create_full_model_from_motion_regression
from wPCC_pipeline.pipelines.prediction.nodes import simulate_euler
from src.models.vmm import VMM
from src.parameters import df_parameters
from src.models.propeller import fit, predict, predictor
from wPCC_pipeline.pipelines.kvlcc2.nodes import fit_propeller_characteristics

In [ ]:
ship="wpcc"
vmm_name = "vmm_martins_simple"
#vmm_name = "vmm_abkowitz"
vmm = catalog.load(vmm_name)
ship_data = catalog.load(f"{ship}.ship_data")
data = catalog.load(f"{ship}.updated.joined.data_ek_smooth")
data['rev'] = data[[r'Prop/PS/Rpm',r'Prop/SB/Rpm']].mean(axis=1)

data['v_p'] = data["v"] + data["r"] * ship_data["x_p"]

added_masses = catalog.load(f"{ship}.added_masses")
exclude_parameters = catalog.load(f"params:{ship}.motion_regression.exclude_parameters")
open_water_characteristics = catalog.load(f"{ship}.open_water_characteristics")
propeller_coefficients = catalog.load(f"{ship}.propeller_coefficients")

In [ ]:
from src.models.propeller import preprocess, lambda_w_p
from src.substitute_dynamic_symbols import run
import statsmodels.api as sm

In [ ]:
data["w_p"] = run(lambda_w_p, inputs=data[['u','rev']], **ship_data, **propeller_coefficients, thrust=data['thrust']/2)
data["delta_w_p"] = data["w_p"] - ship_data['w_p0']

y_p = -0.35
up = data['u'] - y_p*data['r']
data["w_p_PS"] = run(lambda_w_p, inputs={}, **ship_data, **propeller_coefficients, 
                     thrust=data['Prop/PS/Thrust'], 
                     u=up, 
                     rev=data['Prop/PS/Rpm'])
data["delta_w_p_PS"] = data["w_p_PS"] - ship_data['w_p0']


y_p = 0.35
up = data['u'] - y_p*data['r']
data["w_p_SB"] = run(lambda_w_p, inputs={}, **ship_data, **propeller_coefficients, 
                     thrust=data['Prop/SB/Thrust'], 
                     u=up, 
                     rev=data['Prop/SB/Rpm'])
data["delta_w_p_SB"] = data["w_p_SB"] - ship_data['w_p0']


In [ ]:
df_train = data

In [ ]:
add_constant = True
model_pos, model_neg = fit(data=df_train, ship_data=ship_data, propeller_coefficients=propeller_coefficients, add_constant=add_constant) 


In [ ]:
model_pos.summary()

In [ ]:
data_predicted_thrust = predict(model_pos=model_pos, model_neg=model_neg, data=df_train, propeller_coefficients=propeller_coefficients, 
                                ship_data=ship_data)


In [ ]:
fig,ax=plt.subplots()
df_train.plot(y='thrust', ax=ax, label='True')
data_predicted_thrust.plot(y='thrust', ax=ax, label='Prediction')

In [ ]:
for id, df_ in df_train.groupby(by='id', sort=False):
    
    mask = data_predicted_thrust['id'] == id
    df_pred = data_predicted_thrust.loc[mask]
    
    fig,axes=plt.subplots(ncols=2)
    fig.set_size_inches(15,5)
    fig.suptitle(id)
    
    ax = axes[0]
    df_.plot(y=[r'delta_w_p_PS',r'delta_w_p_SB'], label=['PS','SB'], ax=ax)
    #df_.plot(y=[r'Prop/PS/Thrust',r'Prop/SB/Thrust'], label=['PS','SB'], ax=ax)
    ax.set_ylim(-1,0.5)
    
    ax = axes[1]
    df_.plot(y=['v_p'], ax=ax)
    ax.set_ylim(data['v_p'].min(),data['v_p'].max())
    
    ax2 = ax.twinx()
    df_.plot(y=['delta'], style='r-', ax=ax2)
    
    

In [ ]:
df_PS = data.copy()
df_PS['thrust'] = df_PS['Prop/PS/Thrust']
df_PS['rev'] = df_PS[r'Prop/PS/Rpm']
df_SB = data.copy()
df_SB['thrust'] = df_SB['Prop/SB/Thrust']
df_SB['rev'] = df_SB[r'Prop/SB/Rpm']

df = pd.concat([df_PS,df_SB])

In [ ]:
ship_data['y_p'] = -0.35

In [ ]:
def features(df, ship_data: dict, add_constant=False):

    if isinstance(df, pd.DataFrame):
        X = pd.DataFrame(index=df.index.copy())
    else:
        X = pd.DataFrame(index=[0])

    #X["delta"] = df["delta"]

    X["delta**2"] = df["delta"] ** 2
    v_p = df["v"] + df["r"] * ship_data["x_p"]
    
    #X['u'] = df['u']
    u_p = -df["r"] * ship_data["y_p"]
    X['u_p'] = u_p
    X["v_p"] = v_p

    X["v_p**2"] = v_p ** 2

    if add_constant:
        X = sm.tools.add_constant(X, has_constant="add")

    return X


def fit_(
    data: pd.DataFrame,
    ship_data: dict,
    propeller_coefficients: dict,
    add_constant=False,
):

    data = preprocess(
        data, ship_data=ship_data, propeller_coefficients=propeller_coefficients
    )

    data["w_p"] = run(lambda_w_p, inputs=data, **ship_data, **propeller_coefficients)

    mask = data["beta_p"] > 0
    df_pos = data.loc[mask].copy()

    X = features(df_pos, ship_data=ship_data, add_constant=add_constant)
    y = df_pos["w_p"] - ship_data["w_p0"]
    linear_regression_pos = sm.OLS(y, X, hasconst=add_constant)
    model_pos = linear_regression_pos.fit()

    mask = data["beta_p"] <= 0
    df_neg = data.loc[mask].copy()

    X = features(df_neg, ship_data=ship_data, add_constant=add_constant)
    y = df_neg["w_p"] - ship_data["w_p0"]
    linear_regression_neg = sm.OLS(y, X, hasconst=add_constant)
    model_neg = linear_regression_neg.fit()

    return model_pos, model_neg

In [ ]:
add_constant = True
model_pos, model_neg = fit_(data=df_PS, ship_data=ship_data, propeller_coefficients=propeller_coefficients, add_constant=add_constant) 


In [ ]:
model_pos.summary()

In [ ]:
model_neg.summary()

In [ ]:
#df_PS["w_p_PS"] = run(lambda_w_p, inputs=df_PS, **ship_data, **propeller_coefficients)

X = pd.DataFrame(index=df_PS.index)
X = sm.tools.add_constant(X, has_constant="add")
y = df_PS["w_p_PS"] - ship_data["w_p0"]
linear_regression = sm.OLS(y, X, hasconst=False)
result = linear_regression.fit()

In [ ]:
result.summary()

In [ ]:
y.plot()